In [ ]:
import json
import bs4 as bs
import urllib
import requests

def factorTouid(factor_name, species):
    """query factor to get uniprot id and other contents"""
    # factor_name = "FUS"
    # species = "human"
    if factor_name == "T":
        factor_name = "TBXT"
    if species == "human":
        url_query = "https://www.uniprot.org/uniprot/?query={0}+AND+{1}&sort=score" . format(factor_name.upper(), species) # query factor
    elif species == "mouse":
        url_query = "https://www.uniprot.org/uniprot/?query={0}+AND+{1}&sort=score" . format(factor_name.lower(), species) # query factor
    elif species == "zebrafish":
        url_query = "https://www.uniprot.org/uniprot/?query={0}+AND+{1}&sort=score" . format(factor_name.lower(), species) # query factor
        species2 = 'danre'
        
    print(url_query)
    flag = 0
    try:
        sauce = requests.get(url_query, timeout = 30)
        soup = bs.BeautifulSoup(sauce.content,'html.parser')
        table_results = soup.find(id = "results")
        tbody = table_results.find("tbody")
        trs = tbody.find_all("tr")
        tds_true = None
        for tr in trs:
            tds = tr.find_all("td")
            review = tds[3].find("div").get("title")
            entry_name = tds[2].get_text().upper() # e.g. SMC3_HUMAN
            gene_name = tds[5].get_text().upper().replace(" ", "$").replace(",", "$") + "$" # gene symbol alias (SMC3 BAM, BMH, CSPG6, SMC3L1)
            if ("%s_"%(factor_name.upper()) in entry_name or "%s"%(factor_name.upper()) in gene_name) and species2.upper() in entry_name:
                tds_true = tds
                break
                
        if tds_true:     
            return(np.array([td.get_text() for td in tds_true])[[1,2,5]])
        else:
            return(np.array(["None"] * 3))

    except:
        # print(factor_name)
        return(np.array(["None"] * 3))
    
def uidTofasta(uid):
    """download protein aa sequence for given uid"""
    fasta_url = "https://www.uniprot.org/uniprot/{0}.fasta" . format(uid)
    fasta_file_name = "{0}.fasta" . format(uid)
    urllib.request.urlretrieve(fasta_url, fasta_file_name)

In [ ]:
list_zebrafish_factor_annotation = []
for factor in gene_symbol_list :
    print(factor)
    list_zebrafish_factor_annotation.append(np.concatenate([np.array([factor]), factorTouid(factor, "zebrafish")]))


In [ ]:
df_zebrafish_factor_annotation = pd.DataFrame(list_zebrafish_factor_annotation)
df_zebrafish_factor_annotation.columns = ["factor", "uniprot_id", "uniprot_entry", "uniprot_alias"]
df_zebrafish_factor_annotation.to_csv("/mnt/Storage/home/wangyiman/early_embryo_network/analysis/early_expr_tf/early_expr_zebrafish_TR_uniprot.txt", header = True, index = False, sep = "\t")
df_zebrafish_factor_annotation

,factor,uniprot_id,uniprot_entry,uniprot_alias
0,thraa,Q98867,THAA_DANRE,"thraa nr1a1a, thra, thraa1, tra1"
1,matr3l1.1,Q800H6,Q800H6_DANRE,matr3l1.1
2,pms1,B0S640,B0S640_DANRE,pms1
3,znf511,Q7ZZ00,ZN511_DANRE,znf511
4,nr4a1,Q6GMG3,Q6GMG3_DANRE,nr4a1
...,...,...,...,...
1038,nsmce2,B3DIX8,B3DIX8_DANRE,nsmce2 LOC564644
1039,lzts2b,E7EYJ6,E7EYJ6_DANRE,"lzts2b LAPSER1, LZTS2"
1040,zdhhc1,E7F4Z4,ZDHC1_DANRE,zdhhc1 dhhc1
1041,MTERF1,E7FGP4,E7FGP4_DANRE,mterf1


In [ ]:
os.chdir('/mnt/Storage/home/wangyiman/early_embryo_network/analysis/early_expr_tf/fasta_TR')
for uniprot_id in df_zebrafish_factor_annotation.loc[:, "uniprot_id"].values:
    if not os.path.isfile(f"{uniprot_id}.fasta"):
        if uniprot_id != "None":
            uidTofasta(uniprot_id)